### 0. 분석 환경 설정

In [ ]:
# 모듈 불러오기
import os
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.tag import pos_tag
from langdetect import detect_langs
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from soyclustering import SphericalKMeans
from scipy.sparse import csr_matrix
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.svm import SVR
import warnings
import nltk

# 경로 설정
os.chdir(r"C:\Users\GilseungAhn\Desktop\한양대\3. 프로젝트\2. 진행중인 과제\포스트코로나 AI 챌린지")

# 경고 무시하기
warnings.filterwarnings("ignore")

# 데이터 시간 범위 (date_range) 설정: 해외유입확진자 수 데이터가 정상수집된 2020년 4월 13일 이후 데이터만 필터링
date_range = pd.date_range(start = pd.Timestamp(year = 2020, month = 4, day = 13),
                           end = pd.Timestamp(year = 2020, month = 5, day = 5)) 

### 1. 데이터 가공

#### 1.1 일자별 누적 해외유입 확진자 수 데이터

1.1.1 데이터 불러오기

In [ ]:
corona_inflow_df = pd.read_csv("데이터/2. 외부데이터/일자별_누적해외유입확진자수.csv", encoding = "utf8")
corona_inflow_df['일자'] = pd.to_datetime(corona_inflow_df['일자']) # 데이터 타입을 날짜로 변환 

1.1.2 데이터 필터링

In [ ]:
corona_inflow_df = corona_inflow_df[corona_inflow_df['일자'].between(date_range[0], date_range[-1])]

1.1.3 신규확진자 계산

In [ ]:
# i일의 신규확진자 = i일의 누적 확진자 - (i-1)일의 누적 확진자
corona_inflow_df['신규확진자'] = corona_inflow_df['누적확진자'] - corona_inflow_df['누적확진자'].shift(1)

# 첫 행에서 발생하는 결측 제거 
corona_inflow_df.dropna(inplace = True)

1.1.4 데이터 내보내기

In [ ]:
corona_inflow_df.to_csv("데이터/3. 정제데이터/일자별_신규해외유입확진자수.csv", index = False)

#### 1.2 해외증시 데이터

1.2.1 데이터 불러오기

In [ ]:
world_stock_df = pd.read_csv("데이터/2. 외부데이터/일자별_해외지수.csv", encoding = "utf8")

# 데이터 타입을 날짜로 변환
world_stock_df['일자'] = pd.to_datetime(world_stock_df['일자']) 

1.2.2 해외지수별 변동률 계산

In [ ]:
# i일 해외지수 변동률 = (i일 해외 지수 - (i-1)일 해외 지수) / (i-1)일 해외 지수 * 100
world_stock_df['니케이지수_변동률'] = (world_stock_df['니케이지수'] - world_stock_df['니케이지수'].shift(1)) / world_stock_df['니케이지수'].shift(1) * 100
world_stock_df['다우산업지수_변동률'] = (world_stock_df['다우산업지수'] - world_stock_df['다우산업지수'].shift(1)) / world_stock_df['다우산업지수'].shift(1) * 100
world_stock_df['S&P500지수_변동률'] = (world_stock_df['S&P500지수'] - world_stock_df['S&P500지수'].shift(1)) / world_stock_df['S&P500지수'].shift(1) * 100
world_stock_df['상해종합지수_변동률'] = (world_stock_df['상해종합지수'] - world_stock_df['상해종합지수'].shift(1)) / world_stock_df['상해종합지수'].shift(1) * 100

1.2.3 장이 열리지 않은 날의 지수를 이전 값으로 채우기

In [ ]:
# 일자를 인덱스로 정의한 뒤 date_range에 포함되지 않은 일자 추가 및 값을 이전값으로 대체
world_date_range = pd.date_range(start = pd.Timestamp(year = 2020, month = 3, day = 1),
                           end = pd.Timestamp(year = 2020, month = 5, day = 5)) 

world_stock_df = world_stock_df.set_index('일자').reindex(world_date_range).fillna(method = 'ffill').reset_index()

# 일자를 인덱스로 정의하는 과정에서 생긴 index 컬럼을 일자 컬럼으로 재변환
world_stock_df.rename(columns = {'index':'일자'}, inplace = True)
world_stock_df.dropna(inplace = True)

1.2.4 데이터 내보내기

In [ ]:
world_stock_df.to_csv("데이터/3. 정제데이터/일자별_해외지수_변동률포함.csv", encoding = "utf8", index = False)

#### 1.3 뉴스 기사 데이터

1.3.1 뉴스 목록 데이터 불러오기

In [ ]:
# 4월 6일, 4월 29일, 5월 6일에 배포한 NewsList 불러오기
NewsList_0406 = pd.read_excel("데이터/1. 제공데이터/0406_배포/3-1. NewsList.xls")
NewsList_0429 = pd.read_excel("데이터/1. 제공데이터/0429_배포/3-1. NewList.xls")
NewsList_0506 = pd.read_excel("데이터/1. 제공데이터/0506_배포/3-1. NewsList.xls")

1.3.2 뉴스 목록 데이터 병합 및 필터링

In [ ]:
# 4월 6일, 4월 29일, 5월 6일에 배포한 NewsList 행 단위 병합
NewsList = pd.concat([NewsList_0406, NewsList_0429, NewsList_0506], axis = 0, ignore_index = True)

del NewsList_0406, NewsList_0429, NewsList_0506 # 메모리 관리를 위한 데이터 삭제

# 게시일자를 날짜 타입으로 변환
NewsList['게시일자'] = pd.to_datetime(NewsList['게시일자']).dt.date

# 필터링 수행: 시간 기준, 감염병명 기준, 컬럼 기준
NewsList = NewsList[NewsList['게시일자'].between(date_range[0], date_range[-1])] # 분석 범위에 해당하는 데이터만 필터링
NewsList = NewsList[NewsList['감염병명'] == 'COVID-19'] # COVID-19 관련 기사만 가져오기
NewsList = NewsList[['파일명', '제목', '게시일자']]

1.3.3 뉴스 기사 데이터 전처리 및 병합

파일 불러오기

In [ ]:
stemmer = PorterStemmer() # 스테머 초기화

# load_news_files: 파일명에 해당하는 파일의 내용을 불러오고 스테밍을 수행하는 함수
def load_news_files(file):
    try:
        file_content = open("데이터/1. 제공데이터/0406_배포/3-2. Contents/" + file + ".txt", encoding = "utf-8").read()
    except:
        try:
            file_content = open("데이터/1. 제공데이터/0429_배포/3-2. Contents/" + file + ".txt", encoding = "utf-8").read()
        except:
            file_content = open("데이터/1. 제공데이터/0506_배포/3-2. Contents/" + file + ".txt", encoding = "utf-8").read()
    
    file_content = file_content.split('Title :')[1].lower().strip()
    output = ' '.join(list(set([stemmer.stem(w) for w in word_tokenize(file_content) if len(w) >= 2])))
    return output

NewsList['기사내용'] = NewsList['파일명'].apply(load_news_files)

기사 내용을 단어 가방으로 변환

In [ ]:
# 영어단어 목록 가져오기
english_word_list = pd.Series(list(set(nltk.corpus.words.words()) - set(stopwords.words('english')))) # 불용어 제거
english_word_list = english_word_list.apply(stemmer.stem).str.lower().drop_duplicates()
english_word_list = english_word_list[english_word_list.apply(len) >= 2] # 길이가 2이상인 단어만 사용

# 최소 하루에 한 번 이상 사용된 단어만 빈발 단어라고 간주
def check_frequent_english_word(_word, _NewsList_per_day): 
    output = True
    for news in _NewsList_per_day:
        if _word not in news:
            output = False
            break
    
    return output

NewsList_per_day = NewsList.groupby('게시일자')['기사내용'].sum().tolist()
frequent_english_word_list = set(english_word_list[english_word_list.apply(check_frequent_english_word, _NewsList_per_day = NewsList_per_day)])

# preprocessing_article(file): file을 읽어와서 텍스트 내에 있는 영어 단어만 추출하여 전처리한 결과를 반환하는 함수
def preprocessing_article(Article, _frequent_english_word_list):
    # 등장한 단어의 원형 집합 만들기
    occur_word_list = set([stemmer.stem(w) for w in word_tokenize(Article) if len(w) >= 2])
    
    # 등장한 단어 가운데 영어만 추출
    occur_english_word_list = frequent_english_word_list & occur_word_list
    return ' '.join(list(occur_english_word_list))

NewsList['기사내용'] = NewsList['기사내용'].apply(preprocessing_article, _frequent_english_word_list = frequent_english_word_list)
NewsList.dropna(inplace = True) # 기사내용이 없거나, 영어가 아닌 기사는 제거

with open("분석 결과 및 모델/단어목록.pckl", "wb") as f:
    pickle.dump(frequent_english_word_list, f)

단어 - 문서 행렬 생성

In [ ]:
# CountVectorizer를 vectorizer로 인스턴스화 및 초기화
vectorizer = CountVectorizer()

# 기사 내용을 단어 문서 행렬로 변환 (셀: 해당 문서에 해당 단어의 출현 빈도)
X = vectorizer.fit_transform(NewsList['기사내용'])
X = X.toarray()

# 출현한 모든 단어 가져오기
words = vectorizer.get_feature_names()

# 단어-문서 행렬의 데이터 타입 변환 (np.array -> pd.DataFrame)
word_document_matrix = pd.DataFrame(X, columns = words)

del X # 메모리 관리를 위한 데이터 삭제

# 날짜 부착 및 데이터 내보내기
word_document_matrix['게시날짜'] = NewsList['게시일자'].values
word_document_matrix.to_csv("데이터/3. 정제데이터/단어-문서행렬.csv", index = False, encoding = "utf8")

1.3.4 일자별 뉴스기사 수 추출

In [ ]:
# 게시일자에 따른 뉴스기사수 계산
num_news_df = pd.DataFrame(NewsList['게시일자'].value_counts()).sort_values(by = '게시일자').reset_index()

# groupby에 의한 바뀐 컬럼명 재변환 수행
num_news_df.rename(columns = {"게시일자":"뉴스기사수"}, inplace = True)
num_news_df.rename(columns = {"index":"게시일자"}, inplace = True)

# date_range에 있으나 게시일자에 없는 날짜의 뉴스기사 수는 0으로 채우기
num_news_df = num_news_df.set_index('게시일자').reindex(date_range).fillna(0).reset_index().rename(columns = {"index":"게시일자"})

# 일자별_뉴스기사수 내보내기
num_news_df.to_csv("데이터/3. 정제데이터/일자별_뉴스기사수.csv", encoding = "utf8", index = False)

#### 1.4 로밍 데이터

1.4.1 데이터 불러오기 및 행단위 병합

In [ ]:
roming_df_0406 = pd.read_csv("데이터/1. 제공데이터/0406_배포/2. Roaming_data.csv")
roming_df_0429 = pd.read_csv("데이터/1. 제공데이터/0429_배포/2. Roaming_data.csv")
roming_df_0506 = pd.read_excel("데이터/1. 제공데이터/0506_배포/2. Roaming_data.xlsx")

roming_df = pd.concat([roming_df_0406, roming_df_0429, roming_df_0506], axis = 0, ignore_index = True)

del roming_df_0406, roming_df_0429, roming_df_0506 # 메모리 관리를 위한 데이터 삭제

1.4.2 return 컬럼 타입 변경 str -> TimeStamp

In [ ]:
# convert_string_to_datetime: M월 D일 형태의 문자열(s)를 TimeStamp로 변환하는 함수
def convert_string_to_datetime(s): 
    s = str(s)
    year, month, day = int(s[:4]), int(s[4:6]), int(s[6:])
    return pd.Timestamp(year = year, month = month, day = day)

# return 컬럼에 convert_string_to_datetime 함수 적용
roming_df['return'] = roming_df['return'].apply(convert_string_to_datetime)

1.4.3 일자 및 국가에 따른 출입자 수 계산

In [ ]:
# return 컬럼과 iso 컬럼 기준 count 합계 계산
roming_df = roming_df.groupby(['return', 'iso'])['count'].sum().reset_index()

# 등장하지 않은 경우 0으로 채운 뒤, 인덱스 리셋
roming_df = roming_df.pivot('return', 'iso', 'count').fillna(0)
roming_df.reset_index(inplace = True)

1.4.4 입국자 합계 계산 및 데이터 내보내기

In [ ]:
roming_df['입국자_합계'] = roming_df.iloc[:, 1:].sum(axis = 1)
roming_df.to_csv("데이터/3. 정제데이터/일자별_입국자.csv", index = False, encoding = "utf8")

### 2. 기초 데이터 생성

#### 2.1 일자 포맷 통일

In [ ]:
world_stock_df['일자'] = pd.to_datetime(world_stock_df['일자'])
corona_inflow_df['일자'] = pd.to_datetime(corona_inflow_df['일자'])
roming_df['return'] = pd.to_datetime(roming_df['return'])
num_news_df['게시일자'] = pd.to_datetime(num_news_df['게시일자'])

#### 2.2 기초 데이터 초기화

2.2.1 base_df 정의

In [ ]:
base_df = corona_inflow_df[['일자', '신규확진자']]

2.2.2 요일 변수 추가

In [ ]:
# 요일 목록 정의
weekday_list = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

# 일자에 해당하는 요일 변수 추가 (예: 2020년 5월 6일 (수) => Wednesday 컬럼만 1, 나머지는 0)
for i in range(len(weekday_list)):
    weekday = weekday_list[i]
    base_df[weekday] = (base_df['일자'].dt.weekday == i).astype(int)

#### 2.3 타 데이터와 병합

2.3.1 예측 기간 변수 및 병합을 위한 임시 키 생성
- D: 현재 시점으로부터 예측 시점까지의 남은 길이 (D = 1, 2, ..., 14)
- D+k_이전일자: 현재 시점으로부터 D+k일 이전의 일자 (k = 0, 1, 2, 3)

In [ ]:
# D = 1인 레코드만 먼저 생성
D = 1
base_df['D'] = D

base_df['D+0_이전일자'] = base_df['일자'] - pd.DateOffset(days = D)
base_df['D+1_이전일자'] = base_df['일자'] - pd.DateOffset(days = D+1)
base_df['D+2_이전일자'] = base_df['일자'] - pd.DateOffset(days = D+2)
base_df['D+3_이전일자'] = base_df['일자'] - pd.DateOffset(days = D+3) 

N = len(base_df) # D = 1인 레코드의 길이만큼 D = 2, 3, ..., 14도 생성하기 위해 길이 저장

# D = 2, 3, ..., 14인 레코드 생성
for D in range(2, 15):
    # D = 1인 레코드만 복제하여 copy_base_df로 저장
    copy_base_df = base_df.iloc[:N].copy()
    
    # D에 해당하는 D 컬럼 및 D+k_이전일자 컬럼 생성
    copy_base_df['D'] = D
    copy_base_df['D+0_이전일자'] = base_df['일자'] - pd.DateOffset(days = D)
    copy_base_df['D+1_이전일자'] = base_df['일자'] - pd.DateOffset(days = D+1)
    copy_base_df['D+2_이전일자'] = base_df['일자'] - pd.DateOffset(days = D+2)
    copy_base_df['D+3_이전일자'] = base_df['일자'] - pd.DateOffset(days = D+3) 
    
    # base_df에 copy_base_df 병합
    base_df = pd.concat([base_df, copy_base_df], axis = 0, ignore_index = True, sort = True)

# 메모리 관리를 위한 데이터 삭제
del copy_base_df

base_df.dropna(inplace = True)

2.3.2 D+0_이전일자, D+1_이전일자의 신규 및 누적 확진자 부착

In [ ]:
corona_inflow_df.rename({"일자":"코로나일자"}, axis = 1, inplace = True)
for d in range(2):
    # 효과적인 병합을 위한 컬럼명 변경 
    # 신규확진자 -> D+k_이전일자 신규확진자 or D+(k-1)_이전일자 신규확진자 -> D+k_이전일자 신규확진자
    corona_inflow_df.rename({"신규확진자": "D+" + str(d) + "_이전일자_신규확진자",
                            "누적확진자": "D+" + str(d) + "_이전일자_누적확진자",
                             
                            "D+" + str(d-1) + "_이전일자_신규확진자": "D+" + str(d) + "_이전일자_신규확진자",
                            "D+" + str(d-1) + "_이전일자_누적확진자": "D+" + str(d) + "_이전일자_누적확진자"},
                           
                            inplace = True,
                           axis = 1)
    
    base_df = pd.merge(base_df, corona_inflow_df, left_on = 'D+' + str(d) + "_이전일자", right_on = '코로나일자').drop('코로나일자', axis = 1)
    
# 메모리 관리를 위한 데이터 삭제
del corona_inflow_df

2.3.3 D+k_이전일자의 해외증시 부착

In [ ]:
world_stock_df.rename(columns = {'일자':'증시일자'}, inplace = True)
for d in range(4):
    base_df = pd.merge(base_df, world_stock_df, left_on = 'D+' + str(d) + "_이전일자", right_on = '증시일자').drop('증시일자', axis = 1)
    for col in world_stock_df.columns:
        base_df.rename(columns = {col: "D+" + str(d) + "_이전일자_" + col}, inplace = True)

# 메모리 관리를 위한 데이터 삭제
del world_stock_df

2.3.4 D+k_이전일자의 입국자 부착

In [ ]:
for d in range(4):
    base_df = pd.merge(base_df, roming_df, left_on = 'D+' + str(d) + "_이전일자", right_on = 'return').drop('return', axis = 1)
    for col in roming_df.columns:
        base_df.rename(columns = {col: "D+" + str(d) + "_이전일자_" + col}, inplace = True)

# 메모리 관리를 위한 데이터 삭제
del roming_df

2.3.5 D+k_이전일자의 뉴스 개수 데이터 부착

In [ ]:
for d in range(4):
    base_df = pd.merge(base_df, num_news_df, left_on = 'D+' + str(d) + "_이전일자", right_on = '게시일자').drop('게시일자', axis = 1)
    for col in num_news_df.columns:
        base_df.rename(columns = {col: "D+" + str(d) + "_이전일자_" + col}, inplace = True)

# 메모리 관리를 위한 데이터 삭제
del num_news_df

2.3.6 기초 데이터 내보내기

In [ ]:
base_df.to_csv("데이터/3. 정제데이터/기초데이터.csv", encoding = "utf8", index = False)

### 3. 모델 학습

#### 3.1 단어군집 및 단어군집 - 문서 행렬 생성

In [ ]:
# make_wcluster_document_df: 단어 - 문서 행렬을 단어 군집 - 문서 행렬로 변환하는 함수
def make_wcluster_document_df(_word_document_matrix, num_cluster): # _ (prefix)는 기존 데이터와 함수 입력이 충돌하지 않도록 넣은 것    
    # 일자 변수 저장
    date_values = _word_document_matrix['게시날짜'].values   
    _word_document_matrix.drop('게시날짜', axis = 1, inplace = True)
    
    # transpose를 사용하여 단어 - 문서 행렬을 문서 - 단어 행렬로 변환
    document_word_matrix = _word_document_matrix.T
    
    # 군집화 수행 및 군집 결과 사전화 (key: 단어, value: 군집 번호)
    spherical_kmeans = SphericalKMeans(n_clusters = num_cluster, max_iter = 1000, verbose = 0, init = 'similar_cut')    
    word_cluster_label = spherical_kmeans.fit_predict(csr_matrix(document_word_matrix))
    word_cluster_dict = pd.Series(word_cluster_label, index = list(_word_document_matrix.columns)).apply(lambda x:"단어군집_" + str(x+1)).to_dict()    
    
    # 단어로 된 컬럼 이름을 군집 번호로 바꾸기
    _word_document_matrix.rename(columns = word_cluster_dict, inplace = True)
    
    # 각 군집에 속한 단어 등장 횟수 합계 계산 
    wcluster_document_df = pd.DataFrame()
    for x in range(1, num_cluster + 1):
        try:
            column_values = _word_document_matrix["단어군집_" + str(x)].sum(axis = 1).values
        except:
            column_values = _word_document_matrix["단어군집_" + str(x)]
        wcluster_document_df["단어군집_" + str(x)] = column_values
        
    # 기초 데이터와 병합을 위한 게시 날짜 변수 추가
    wcluster_document_df['게시날짜'] = date_values
    
    # 날짜별 단어_군집 변수 합계 계산
    cluster_columns = ["단어군집_" + str(x) for x in range(1, num_cluster + 1)]
    wcluster_document_df = wcluster_document_df.groupby('게시날짜', as_index = False)[cluster_columns].sum()
    
    # 각 군집에 속한 단어 등장 비율 계산 
    wcluster_document_df[cluster_columns] = wcluster_document_df[cluster_columns].values / wcluster_document_df[cluster_columns].sum(axis = 1).values.reshape(len(wcluster_document_df), 1)
        
    return word_cluster_dict, wcluster_document_df

# 단어군집 및 단어군집 - 문서 행렬 생성 
word_document_matrix = pd.read_csv("데이터/3. 정제데이터/단어-문서행렬.csv", encoding = "utf8")
base_df = pd.read_csv("데이터/3. 정제데이터/기초데이터.csv", encoding = "utf8")
num_cluster = 40
word_cluster_dict, wcluster_document_df = make_wcluster_document_df(word_document_matrix, num_cluster)

#### 3.2 학습 데이터 생성

In [ ]:
def generate_training_data(_wcluster_document_df, _base_df, K): 
    _wcluster_document_df['게시날짜'] = pd.to_datetime(_wcluster_document_df['게시날짜'])
    for d in range(4):
        _base_df["D+" + str(d) + "_이전일자"] = pd.to_datetime(_base_df["D+" + str(d) + "_이전일자"])
        _base_df = pd.merge(_base_df, _wcluster_document_df, left_on = "D+" + str(d) + "_이전일자", right_on = '게시날짜').drop('게시날짜', axis = 1)
        for col in _wcluster_document_df.columns:
            _base_df.rename(columns = {col:"D+" + str(d) + "_이전일자_" + col}, inplace = True)

    _base_df.sort_values(by = '일자', inplace = True)
    day_list = _base_df['일자'].unique()
    
    train_df = _base_df
    
    sample_weight = np.array(np.exp(K * np.arange(len(day_list)) / len(day_list)), dtype = int) - 1    
    sample_weight_dict = pd.Series(sample_weight, index = day_list).to_dict()
    for day, weight in sample_weight_dict.items():
        if weight > 0:
            duplicated_samples = train_df[train_df['일자'] == day]
            train_df = train_df.append([duplicated_samples] * weight, ignore_index = True)

    X = train_df.drop(['신규확진자', '일자', 'D+0_이전일자', 'D+1_이전일자', 'D+2_이전일자', 'D+3_이전일자'], axis = 1)       
    Y = train_df['신규확진자']    
    
    return X, Y

K = 2
X, Y = generate_training_data(wcluster_document_df, base_df, K)

#### 3.3 특징 선택 수행

In [ ]:
k = 300
feature_selector = SelectKBest(f_regression, k = k).fit(X, Y)
selected_features = list(X.columns[feature_selector.get_support()])
selected_features = list(set(selected_features + ['D', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']))

#### 3.4 최종 모델 정의 및 학습

In [ ]:
model = SVR(C = 4, degree = 1, epsilon = 1, gamma = 0.1, kernel = 'rbf', max_iter = 10000)
model.fit(X[selected_features], Y)

#### 3.5 학습 결과 내보내기

In [ ]:
# 최종 군집, 특징 집합, 모델 내보내기
with open("분석 결과 및 모델/최종_군집.pckl", "wb") as f:
    pickle.dump(word_cluster_dict, f)

with open("분석 결과 및 모델/최종_특징집합.pckl", "wb") as f:
    pickle.dump(selected_features, f)
    
with open("분석 결과 및 모델/최종_모델.pckl", "wb") as f:
    pickle.dump(model, f)

### 4. 예측 수행

#### 4.1 예측 시점 정의

In [ ]:
# 5월 2일 ~ 5일 데이터를 활용하여, 5월 6일 이후 2주 예측
input_date_range = pd.date_range(start = pd.Timestamp(year = 2020, month = 5, day = 2),
                           end = pd.Timestamp(year = 2020, month = 5, day = 5)) 

#### 4.2 데이터 불러오기 및 필터링

4.2.1 누적해외유입확진자 수

In [ ]:
# 데이터 불러오기
corona_inflow_df = pd.read_csv("데이터/2. 외부데이터/일자별_누적해외유입확진자수.csv", encoding = "utf8")

# 데이터 타입을 날짜로 변환
corona_inflow_df['일자'] = pd.to_datetime(corona_inflow_df['일자'])  

# 신규확진자 계산
corona_inflow_df['신규확진자'] = corona_inflow_df['누적확진자'] - corona_inflow_df['누적확진자'].shift(1)

# 기간 필터링
corona_inflow_df = corona_inflow_df[corona_inflow_df['일자'].between(input_date_range[0], input_date_range[-1])]

4.2.2 해외 증시 데이터

In [ ]:
# 데이터 불러오기
world_stock_df = pd.read_csv("데이터/2. 외부데이터/일자별_해외지수.csv", encoding = "utf8")
world_stock_df['일자'] = pd.to_datetime(world_stock_df['일자']) # 데이터 타입을 날짜로 변환 

# 변동률 계산
world_stock_df['니케이지수_변동률'] = (world_stock_df['니케이지수'] - world_stock_df['니케이지수'].shift(1)) / world_stock_df['니케이지수'].shift(1) * 100
world_stock_df['다우산업지수_변동률'] = (world_stock_df['다우산업지수'] - world_stock_df['다우산업지수'].shift(1)) / world_stock_df['다우산업지수'].shift(1) * 100
world_stock_df['S&P500지수_변동률'] = (world_stock_df['S&P500지수'] - world_stock_df['S&P500지수'].shift(1)) / world_stock_df['S&P500지수'].shift(1) * 100
world_stock_df['상해종합지수_변동률'] = (world_stock_df['상해종합지수'] - world_stock_df['상해종합지수'].shift(1)) / world_stock_df['상해종합지수'].shift(1) * 100

# Null값 채우기
date_range = pd.date_range(start= world_stock_df['일자'].min(), end = world_stock_df['일자'].max())
world_stock_df = world_stock_df.set_index('일자').reindex(date_range).fillna(method = 'ffill').reset_index()
world_stock_df.rename(columns = {'index':'일자'}, inplace = True)

# 기간 필터링
world_stock_df = world_stock_df[world_stock_df['일자'].between(input_date_range[0], input_date_range[-1])]

4.2.3 뉴스 데이터

In [ ]:
# make_wcluster_document_df_based_on_previous_cluster_model: 이전 군집 결과를 바탕으로 단어 - 군집 행렬을 생성하는 함수
def make_wcluster_document_df_based_on_previous_cluster_model(_word_document_matrix, word_cluster_dict): # _ (prefix)는 기존 데이터와 함수 입력이 충돌하지 않도록 넣은 것    
    # 일자 변수 저장
    date_values = _word_document_matrix['게시날짜'].values          

    # 단어로 된 컬럼 이름을 군집 번호로 바꾸기
    _word_document_matrix.rename(columns = word_cluster_dict, inplace = True)
    
    # 각 군집에 속한 단어 등장 횟수 합계 계산 
    wcluster_document_df = pd.DataFrame()
    num_cluster = len(set(word_cluster_dict.values()))
    for x in range(1, num_cluster + 1):
        try:
            column_values = _word_document_matrix["단어군집_" + str(x)].sum(axis = 1).values
        except:
            column_values = _word_document_matrix["단어군집_" + str(x)]
        wcluster_document_df["단어군집_" + str(x)] = column_values
            
    # 기초 데이터와 병합을 위한 게시 날짜 변수 추가
    wcluster_document_df['게시날짜'] = date_values
    
    # 날짜별 단어_군집 변수 합계 계산
    cluster_columns = ["단어군집_" + str(x) for x in range(1, num_cluster + 1)]
    wcluster_document_df = wcluster_document_df.groupby('게시날짜', as_index = False)[cluster_columns].sum()
    
    # 각 군집에 속한 단어 등장 비율 계산 
    wcluster_document_df[cluster_columns] = wcluster_document_df[cluster_columns].values / wcluster_document_df[cluster_columns].sum(axis = 1).values.reshape(len(wcluster_document_df), 1)
        
    return wcluster_document_df

In [ ]:
# 데이터 불러오기
NewsList = pd.read_excel("데이터/1. 제공데이터/0506_배포/3-1. NewsList.xls")

# 데이터 타입을 날짜로 변환 (시간 제외)
NewsList['게시일자'] = pd.to_datetime(NewsList['게시일자']).dt.date

# COVID-19 관련 기사만 가져오기
NewsList = NewsList[NewsList['감염병명'] == 'COVID-19'] 

# 기간 및 컬럼 필터링
NewsList = NewsList[['파일명', '제목', '게시일자']]
NewsList = NewsList[NewsList['게시일자'].between(input_date_range[0], input_date_range[-1])]

# 뉴스기사 병합
NewsList['기사내용'] = NewsList['파일명'].apply(load_news_files)
NewsList.dropna(inplace = True)

# 뉴스기사 전처리
# 단어 목록 가져오기
with open("분석 결과 및 모델/단어목록.pckl", "rb") as f:
    frequent_english_word_list = pickle.load(f)

NewsList['기사내용'] = NewsList['기사내용'].apply(preprocessing_article, _frequent_english_word_list = frequent_english_word_list)

# 단어 - 문서 행렬로 변환
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(NewsList['기사내용'])
X = X.toarray()
words = vectorizer.get_feature_names()
word_document_matrix = pd.DataFrame(X, columns = words)

# 최종 군집 사전 가져오기
with open("분석 결과 및 모델/최종_군집.pckl", "rb") as f:
    word_cluster_dict = pickle.load(f)
    
# 단어 목록 가져오기
model_word_list = list(word_cluster_dict.keys())    

model_word_list = list(set(model_word_list) & set(word_document_matrix.columns))   
word_document_matrix = word_document_matrix[model_word_list]

# 게시 날짜 부착
word_document_matrix['게시날짜'] = NewsList['게시일자'].values

# 단어 군집 - 문서 행렬로 변환
wcluster_document_df = make_wcluster_document_df_based_on_previous_cluster_model(word_document_matrix, word_cluster_dict)
wcluster_document_df['게시날짜'] = pd.to_datetime(wcluster_document_df['게시날짜'])

4.2.4 로밍 데이터

In [ ]:
roming_df = pd.read_excel("데이터/1. 제공데이터/0506_배포/2. Roaming_data.xlsx")
roming_df['return'] = roming_df['return'].apply(convert_string_to_datetime)
roming_df = roming_df.groupby(['return', 'iso'])['count'].sum().reset_index()
roming_df = roming_df.pivot('return', 'iso', 'count').fillna(0)
roming_df.reset_index(inplace = True)
roming_df['입국자_합계'] = roming_df.iloc[:, 1:].sum(axis = 1)

# 필터링 수행
# training_roming_df_cols: 학습 로밍 데이터에 포함된 국가 목록
training_roming_df_cols = open("데이터/3. 정제데이터/일자별_입국자.csv", encoding = "utf8").readline().split(',')
training_roming_df_cols[-1] = training_roming_df_cols[-1][:-1]

roming_df = roming_df[roming_df['return'].between(input_date_range[0], input_date_range[-1])]

for col in training_roming_df_cols:
    if col not in roming_df.columns:
        roming_df[col] = 0

roming_df = roming_df[training_roming_df_cols]

4.2.5 뉴스기사 수

In [ ]:
day_num_News = pd.DataFrame(NewsList['게시일자'].value_counts()).sort_values(by = '게시일자').reset_index()
day_num_News.rename(columns = {"게시일자":"뉴스기사수"}, inplace = True)
day_num_News.rename(columns = {"index":"게시일자"}, inplace = True)
day_num_News = day_num_News.set_index('게시일자').reindex(input_date_range).fillna(0).reset_index().rename(columns = {"index":"게시일자"})

#### 4.3 학습 데이터 생성

In [ ]:
# 학습 데이터 초기화
prediction_time = []
D = []
for i in range(1, 15):
    prediction_time.append(input_date_range[-1] + pd.Timedelta(i, unit = 'd'))
    D.append(i)

train_df = pd.DataFrame({"일자":prediction_time, "D": D})

# 요일 변수 추가
weekday_list = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
for i in range(len(weekday_list)):
    weekday = weekday_list[i]
    train_df[weekday] = (train_df['일자'].dt.weekday == i).astype(int)

# 예측 기간 변수 추가  
train_df['D+0_이전일자'] = input_date_range[-1]
train_df['D+1_이전일자'] = input_date_range[-1] - pd.DateOffset(days = 1)
train_df['D+2_이전일자'] = input_date_range[-1] - pd.DateOffset(days = 2)
train_df['D+3_이전일자'] = input_date_range[-1] - pd.DateOffset(days = 3) 

# D-0, D-1 시점의 신규, 누적 확진자 부착
for d in range(2):
    date = train_df['D+' + str(d) + '_이전일자'].iloc[0]
    train_df['D+' + str(d) + '_이전일자_신규확진자'] = corona_inflow_df[corona_inflow_df['일자'] == date].iloc[0, 2]
    train_df['D+' + str(d) + '_이전일자_누적확진자'] = corona_inflow_df[corona_inflow_df['일자'] == date].iloc[0, 1]

# D-0, D-1, D-2, D-3 시점의 해외증시, 입국자, 뉴스 국가 출현 데이터 부착
for d in range(4):
    date = train_df['D+' + str(d) + '_이전일자'].iloc[0]    
    for c in range(1, len(world_stock_df.columns)):
        train_df['D+' + str(d) + '_이전일자_' + world_stock_df.columns[c]] = world_stock_df[world_stock_df['일자'] == date].iloc[0, c]
    
    for c in range(1, len(roming_df.columns)):
        train_df['D+' + str(d) + '_이전일자_' + roming_df.columns[c]] = roming_df[roming_df['return'] == date].iloc[0, c]

    for c in range(1, len(wcluster_document_df.columns)):
        train_df['D+' + str(d) + '_이전일자_' + wcluster_document_df.columns[c]] = wcluster_document_df[wcluster_document_df['게시날짜'] == date].iloc[0, c]
        
    train_df['D+' + str(d) + '_이전일자_' + '뉴스기사수'] = day_num_News[day_num_News['게시일자'] == date].iloc[0, 1]

#### 4.4 모델 불러오기 및 예측

4.4.1 모델 불러오기

In [ ]:
with open("분석 결과 및 모델/최종_특징집합.pckl", "rb") as f:
    selected_features = pickle.load(f)

with open("분석 결과 및 모델/최종_모델.pckl", "rb") as f:
    model = pickle.load(f)

4.4.2 예측 수행

In [ ]:
X = train_df.copy()
X = X.drop(['일자', 'D+0_이전일자', 'D+1_이전일자', 'D+2_이전일자', 'D+3_이전일자'], axis = 1)

X = X[selected_features]
Y = model.predict(X)
Y[Y < 0] = 0
Y = np.round(Y, 0).astype(int)
#Y = np.array(Y, dtype = int)

#### 4.5 예측 결과 보여주기

In [ ]:
pd.Series(Y, index = train_df['일자'])